In [5]:
import random
import numpy as np
import matplotlib.pyplot as plt

from scipy import signal
# signal module from scipy performs operations on 2D matrices 
# (typically used for image processing, filtering, and feature extraction)

from keras.datasets import mnist
from tensorflow.keras import utils

### Base layer class to specify the Layer properites

In [6]:
class Layer:

    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        pass

    def backward(self, output_gradient, learning_rate):
        pass

### Forward Propagation in Convolution Layer

In [ ]:
class Convolutional(Layer):
    
    def __init__(self, input_shape, kernel_size, depth):
        # Input_shape is 3 dimensional (dxhxw)
        # input depth  = no.of channels
        # input_height = image height and
        # input_width  = image width
        input_depth, input_height, input_width = input_shape

        # Depth represents the number of kernels of our convolutional layer
        self.depth = depth
        self.input_shape = input_shape

        # number of channels in the image
        self.input_depth = input_depth

        # Calculating Conv layer output of 3 dimensions
        # first dim  = number of filters/kernels
        # second dim = height of the output matrix after applying convolution
        # third dim  = width of the output matrix after applying convolution
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)

        # Kernels shape takes 4 dimensions
        # depth = no. of kernels
        # input_depth = image channels
        # kernel_size = kernel dimension
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)

        # Initalizing the Kernels weights randomly
        self.kernels = np.random.randn(*self.kernels_shape)

        # Initializing the biases randomly
        self.biases = np.random.rand(*self.output_shape)

    # Forward Pass
    def forward(self, input):
        self.input = input
        # Inititialize output matrix with output_shape
        self.output = np.zeros(self.output_shape)

        # Nested loop for traversing across all filters (depth), then all channels (input_depth) in every input image
        for i in range(self.depth):
            for j in range(self.input_depth):
                # Output = Conv(Input, Kernel) + Bias
                self.output[i] = self.biases[i] + signal.correlate2d(self.input[j], self.kernels[i, j], "valid")    
                                                                                  # valid stands for no padding
        return self.output

    # Backward Pass
    def backward(self, output_gradient, learning_rate):
        # Intializing the gradient of the kernels as zeros
        kernels_gradient = np.zeros(self.kernels_shape)

        # Intializing the gradient of the input as zeros
        input_gradient = np.zeros(self.input_shape)

        # Nested loop for updating the gradients of kernels and inputs, 
        # first traversing all filters (depth), then all channels (input_depth) in every input image
        for i in range(self.depth):
            for j in range(self.input_depth):
                # Calculate kernels gradient in every i and j index in the kernel, 
                kernels_gradient[i,j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")  
                                        # Computes the cross-correlation between two 2D arrays

                #Calculate input gradient by sliding the kernel on the output gradient matrix
                input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i, j], "full")  
                                        # Performs 2D convolution but flips the kernel before sliding over the input
                                        # full stands for full padding
                                        # padding = kernel size−1

        # Update the kernels and biases w.r.t. learned features (stored in gradients)
        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * np.sum(output_gradient)

        return input_gradient